In [1]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup
import pandas as pd
import json #permet de convertir le JAVA en python et vise-versa
from datetime import datetime
import requests #permet de faire des requêtes http


In [2]:
list_cities = ["Mont Saint Michel", "St Malo","Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille",
               "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon",
               "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas" ,"Cassis", "Marseille",
               "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer",
               "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [3]:
#On va chercher les coordonées GPS pour toutes les villes, que l'on regroupe dans list_gps :

list_gps = []

for element in list_cities:
    r_gps = requests.get("https://nominatim.openstreetmap.org/search?q={}&format=json".format(element)) #API request
    dic_gps = r_gps.json()
    lat = float(dic_gps[0]["lat"])
    lon = float(dic_gps[0]["lon"])
    list_gps.append([lat,lon])

In [4]:
#On crée dataset vide, avec les noms colonnes :

columns = ["city_name", "city_id", "latitude", "longitude", "day_month", "weather_desc",
         "prob_precipitation", "daily_temperature"]

dataset = pd.DataFrame(columns = columns)
dataset

,city_name,city_id,latitude,longitude,day_month,weather_desc,prob_precipitation,daily_temperature


In [5]:
key_api = "a86cbdb4c7a66a1a2f4a7905567ef57f"

    
for j in range(len(list_cities)):
    
    city = list_cities[j]
    city_id = j
    lat = list_gps[j][0]
    lon = list_gps[j][1]
    
    r_weather = requests.get(
    "https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&units={}&exclude=minutely,hourly&appid={}"\
        .format(lat, lon, 'metric', key_api)) 
    
    
    dic_weather = r_weather.json()

 
    dictionnary={'city_name': city,
                 'city_id':city_id,
                 'latitude':lat,
                 'longitude':lon}
    
    for i in range(1,8):
        timestamp = dic_weather["daily"][i]["dt"]
        date = datetime.fromtimestamp(timestamp)
        dictionnary['day_month'] = date.strftime("%d/%m")
        dictionnary['weather_desc'] = dic_weather["daily"][i]["weather"][0]["description"]
        dictionnary['daily_temperature'] = dic_weather["daily"][i]['temp']['day']
        dictionnary['prob_precipitation'] = float(dic_weather["daily"][i]["pop"])
        dataset = dataset.append(dictionnary, ignore_index = True)

In [6]:
dataset.head(10)

,city_name,city_id,latitude,longitude,day_month,weather_desc,prob_precipitation,daily_temperature
0,Mont Saint Michel,0,48.635523,-1.510257,21/09,few clouds,0.00,19.96
1,Mont Saint Michel,0,48.635523,-1.510257,22/09,few clouds,0.00,19.24
2,Mont Saint Michel,0,48.635523,-1.510257,23/09,broken clouds,0.00,18.50
3,Mont Saint Michel,0,48.635523,-1.510257,24/09,clear sky,0.00,18.28
4,Mont Saint Michel,0,48.635523,-1.510257,25/09,light rain,0.54,20.29
5,Mont Saint Michel,0,48.635523,-1.510257,26/09,light rain,0.93,19.44
6,Mont Saint Michel,0,48.635523,-1.510257,27/09,broken clouds,0.04,16.77
7,St Malo,1,48.649518,-2.026041,21/09,scattered clouds,0.00,17.38
8,St Malo,1,48.649518,-2.026041,22/09,few clouds,0.00,16.92
9,St Malo,1,48.649518,-2.026041,23/09,scattered clouds,0.00,17.37


In [7]:
# On calcul la météo moyenne pour la semaine à venir :

mean_temp = dataset.groupby("city_name")["daily_temperature"]\
                .mean()\
                .reset_index()

mean_temp.rename(columns = {"city_name": "city_name", 
                          "daily_temperature": "average_temperature"}, 
                 inplace = True)
mean_temp

,city_name,average_temperature
0,Aigues Mortes,23.641429
1,Aix en Provence,24.820000
2,Amiens,18.374286
3,Annecy,19.448571
4,Ariege,15.335714
5,Avignon,24.838571
6,Bayeux,18.428571
7,Bayonne,20.487143
8,Besancon,18.087143
9,Biarritz,19.745714


In [8]:
dataset = dataset.merge(mean_temp, on = "city_name", how = "inner")
dataset

,city_name,city_id,latitude,longitude,day_month,weather_desc,prob_precipitation,daily_temperature,average_temperature
0,Mont Saint Michel,0,48.635523,-1.510257,21/09,few clouds,0.00,19.96,18.925714
1,Mont Saint Michel,0,48.635523,-1.510257,22/09,few clouds,0.00,19.24,18.925714
2,Mont Saint Michel,0,48.635523,-1.510257,23/09,broken clouds,0.00,18.50,18.925714
3,Mont Saint Michel,0,48.635523,-1.510257,24/09,clear sky,0.00,18.28,18.925714
4,Mont Saint Michel,0,48.635523,-1.510257,25/09,light rain,0.54,20.29,18.925714
...,...,...,...,...,...,...,...,...,...
240,La Rochelle,34,46.159113,-1.152043,23/09,clear sky,0.00,20.54,19.598571
241,La Rochelle,34,46.159113,-1.152043,24/09,clear sky,0.00,19.76,19.598571
242,La Rochelle,34,46.159113,-1.152043,25/09,light rain,0.20,18.44,19.598571
243,La Rochelle,34,46.159113,-1.152043,26/09,light rain,0.68,20.41,19.598571


In [9]:
dataset.groupby(["city_id"], axis=1)

In [11]:
# On sélectionne les 3 villes avec la meilleure météo pour la semaine et on la store dans :
dataset.to_csv('villes - Projection meteo.csv', index = False)